In [1]:
import pandas as pd
import sklearn
import librosa
import numpy as np
from glob import glob
from sklearn.preprocessing import MultiLabelBinarizer
from IPython.display import Audio
from audioread.exceptions import NoBackendError
import sys
import resampy

In [2]:
sound_list = glob('./fma/data/fma_medium/*/*.mp3')

### Note enough room on my computer to load the entire dataset + data augmentation
##### Update: 5k datapoints still too much, maxed out ram.

In [3]:
sound_list = sound_list[:1999]

### Preprocessing

In [4]:
tracks = pd.read_csv('./fma/data/fma_metadata/tracks.csv', low_memory=False)
genres = pd.read_csv('./fma/data/fma_metadata/genres.csv', low_memory=False)

In [5]:
tracks.rename(columns=tracks.iloc[0], inplace=True)
tracks.rename(columns={tracks.columns[0]: tracks.iloc[1][0] }, inplace = True)
tracks = tracks.iloc[2:]

genres = genres[["genre_id", "title"]]
genres.set_index(genres.columns[0], inplace=True)
glist = genres
genres = genres.to_dict()
genres = genres['title']

In [6]:
def transform_genre_col(x):
    remove_chars = ["[", "]", " "]
    for char in remove_chars:
        x = x.replace(char, "")
    if x == "":
        return None
    if "," in x:
        x = x.split(',')
    else:
        x = [x]
    lst = []
    for g in x:
        lst.append(genres[int(g)])
    return lst

def transform_id_col(x):
    return x.zfill(6)

def extract_id(x):
    return x[26:-4]

def check_list(x):
    return any(elem in x for elem in rmls)

def get_list(x):
    if any(elem in x for elem in rmls):
        c = x
        for e in rmls:
            if e in x:
                c.remove(e)
        return c
    return 0

def check_null(x):
    return x == []

data = tracks[["track_id", "bit_rate", "genre_top", "genres", "genres_all"]]
data.loc[:,"track_id"] = data["track_id"].apply(transform_id_col)
result = list(map(extract_id,sound_list))
data = data.loc[data.track_id.isin(result)]
data.loc[:,"genres"] = data['genres'].apply(transform_genre_col)
data.loc[:,"genres_all"] = data['genres_all'].apply(transform_genre_col)
data['all_genres'] = data.apply(lambda x: list(set(x['genres']).union(set(x['genres_all']), set([x["genre_top"]]))), axis=1)
data.drop(columns=['genre_top','genres','genres_all'], inplace=True)

enc = MultiLabelBinarizer()
encgenres = pd.DataFrame(enc.fit_transform(data["all_genres"]))
classes = []

for i in range(len(enc.classes_)):
    classes.append([i,len(encgenres.loc[encgenres[i]==1])])
    
rmls = []
for i in range(len(enc.classes_)):
    
    if classes[i][1] < 101:
        rmls.append(enc.classes_[i])

r = data["all_genres"].apply(check_list)
c = data["all_genres"].apply(get_list)
n = data["all_genres"].apply(check_null)

data.drop(data.loc[n].index, inplace=True)

### Data trimmed because I literally couldn't load everything. 32gb ram not enough :(

In [7]:
data

,track_id,bit_rate,all_genres
2,000002,256000,[Hip-Hop]
3,000003,256000,[Hip-Hop]
4,000005,256000,[Hip-Hop]
11,000134,256000,[Hip-Hop]
13,000136,256000,[Rock]
...,...,...,...
6999,012381,320000,[Hip-Hop]
7002,012388,320000,[Electronic]
7006,012392,320000,[Electronic]
7008,012394,320000,[Hip-Hop]


In [8]:
enc = MultiLabelBinarizer()
encgenres = pd.DataFrame(enc.fit_transform(data["all_genres"]))
classes = []

for i in range(len(enc.classes_)):
    classes.append([i,len(encgenres.loc[encgenres[i]==1])])

In [9]:
for l in sorted(classes, key=lambda x: x[1], reverse=True):
    print(f"{enc.classes_[l[0]]}: {l[1]}")

Rock: 782
Folk: 241
Punk: 214
Hip-Hop: 195
Electronic: 186
Experimental: 157
International: 112
Indie-Rock: 109


### Big class imbalance, need to augment data

First need to add audio data to the frame

In [10]:
srs = []
ys = []
bad_files = []
for i in range(len(sound_list)):
    print(i)
    try:
        y, sr = librosa.load(sound_list[i], res_type='soxr_qq', sr=16000)
    except NoBackendError as e:
        bad_files.append(i)
    ys.append(y)
    srs.append(sr)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

C:\Users\miles\AppData\Local\Temp\ipykernel_13092\1865537184.py:7: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(sound_list[i], res_type='soxr_qq', sr=16000)
C:\Users\miles\school\cs5850\Genre Classification\project\env\lib\site-packages\librosa\core\audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


318
319
320
321
322
323
324
325
326
327
328
329
330
331
332
333
334
335
336
337
338
339
340
341
342
343
344
345
346
347
348
349
350
351
352
353
354
355
356
357
358
359
360
361
362
363
364
365
366
367
368
369
370
371
372
373
374
375
376
377
378
379
380
381
382
383
384
385
386
387
388
389
390
391
392
393
394
395
396
397
398
399
400
401
402
403
404
405
406
407
408
409
410
411
412
413
414
415
416
417
418
419
420
421
422
423
424
425
426
427
428
429
430
431
432
433
434
435
436
437
438
439
440
441
442
443
444
445
446
447
448
449
450
451
452
453
454
455
456
457
458
459
460
461
462
463
464
465
466
467
468
469
470
471
472
473
474
475
476
477
478
479
480
481
482
483
484
485
486
487
488
489
490
491
492
493
494
495
496
497
498
499
500
501
502
503
504
505
506
507
508
509
510
511
512
513
514
515
516
517
518
519
520
521
522
523
524
525
526
527
528
529
530
531
532
533
534
535
536
537
538
539
540
541
542
543
544
545
546
547
548
549
550
551
552
553
554
555
556
557
558
559
560
561
562
563
564
565
566
567


In [11]:
dic = {'track_id':result, 'y':ys, 'sr': srs}

In [12]:
df = pd.DataFrame(dic)

In [13]:
big_data = data.merge(df, on="track_id", how="inner")

In [14]:
### Make the original data OOO
def rename_id(x):
    return "OOO-"+x
big_data.loc[:,"track_id"] = big_data["track_id"].apply(rename_id)

In [15]:
big_data

,track_id,bit_rate,all_genres,y,sr
0,OOO-000002,256000,[Hip-Hop],"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",16000
1,OOO-000003,256000,[Hip-Hop],"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",16000
2,OOO-000005,256000,[Hip-Hop],"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",16000
3,OOO-000134,256000,[Hip-Hop],"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",16000
4,OOO-000136,256000,[Rock],"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",16000
...,...,...,...,...,...
1668,OOO-012381,320000,[Hip-Hop],"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",16000
1669,OOO-012388,320000,[Electronic],"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",16000
1670,OOO-012392,320000,[Electronic],"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",16000
1671,OOO-012394,320000,[Hip-Hop],"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",16000


In [16]:
# Augmentations

def aug_noise(y, scale):
    if scale > 0.5:
        sys.exit("Scale too high on aug_noise")
    noise = np.random.normal(0, y.std(), y.size)
    return y + noise*scale

def aug_pitch(y, sr, scale):
    if scale > 4 or scale < -4:
        sys.exit("Scale out of bounds on aug_pitch")
    return librosa.effects.pitch_shift(y, sr=sr, n_steps=scale)

def aug_polarity(y):
    return y*-1

def aug_gain(y, scale):
    if scale > 5 or scale < 0.6:
        sys.exit("Scale out of bounds on aug_pitch")
    return y * scale

aug_list = [aug_polarity, aug_noise, aug_pitch, aug_gain]
aug_num = [1, 5, 8, 9]


# Augment
def calc_augments(required_sets):
    '''
    Formula for number of resulting sets:
    2**n + 16*i
    where n is the number of types of augmentations
    where i is the number of augmentations after the first 4 counting for n
    It will always incremend n first, as that increases the number of datapoints the fastest
    '''
    augs_left = aug_num
    transforms = []
    total_sets = 1

    for i in range(1,5):
        transforms.append(aug_list[i-1])
        if 2**i >= np.ceil(required_sets):
            return transforms
    required_sets-=16
    
    rs = required_sets / 16
    pool = aug_list.copy()
    pool.remove(aug_polarity)
    for i in range(int(np.ceil(rs))):
        cn = transforms.count(aug_noise)
        cp = transforms.count(aug_pitch)
        cg = transforms.count(aug_gain)
        
        if (cg < cp and cg <=8) or (cp==8 and cg <=8):
            transforms.append(aug_list[3])
        elif (cp < cn and cp <= 7) or (cn==5 and cp <=7):
            transforms.append(aug_list[2])
        elif cn <= 4:
            transforms.append(aug_list[1])
        else:
            sys.exit("Not enough augmentation to reach target")
            
    return transforms
    

def get_genre(x, genre):
    return genre in x

def get_args(transforms):
    
    transform_list = [0,0,0,0]
    augments = []
    for augment in transforms:
        
        if augment == aug_polarity:
            
            transform_list[0] += 1
            if transform_list[0] == 2:
                sys.exit("Too many aug_polarity")
            
            augments.append([aug_polarity,[]])
            
        if augment == aug_noise:
            
            transform_list[1] += 1
            if transform_list[1] == 6:
                sys.exit("Too many aug_noise")
            
            augments.append([aug_noise, [0.1*transform_list[1]]])
        
        if augment == aug_pitch:
            
            transform_list[2] += 1
            if transform_list[2] == 9:
                sys.exit("Too many aug_pitch")
            
            augments.append([aug_pitch, [(-1)**(transform_list[2]+1)*np.ceil(transform_list[2]/2)]])
        
        if augment == aug_gain:
            
            transform_list[3] += 1
            if transform_list[3] == 10:
                sys.exit("Too many aug_gain")
            
            if transform_list[3] % 2 == 1:
                augments.append([aug_gain, [(0.5*transform_list[3])+1.5]])
            else:
                augments.append([aug_gain, [((-1/20)*transform_list[3])+1]])
    
    return augments

def apply_augmentations(augs, ind):
    for i, aug in enumerate(augs):
        if index == []:
            index.append(str(0).zfill(3)+ind.track_id[3:])
        else:
            index.append(str(int(index[-1][:3])+1).zfill(3)+ind.track_id[3:])
        if aug[0] == aug_pitch:
            augmented.append(aug[0](ind.y,ind.sr, *aug[1]))
        else:
            augmented.append(aug[0](ind.y, *aug[1]))
        if i < len(augs) -1:
            apply_augmentations(augs[i+1:], ind)
    

def augment_data(genre, target):
    gs = big_data.loc[big_data["all_genres"].apply(get_genre, args=(genre,))]
    current = len(gs)
    required_sets = target/current
    
    transforms = calc_augments(required_sets)
    augments = get_args(transforms)
    
    big_test = pd.DataFrame()
    for i in range(len(gs)):
        
        if current+len(big_test) >= target:
            return big_test
        
        print(f"{genre}: {(current+len(big_test))/target}")
        apply_augmentations(augments, gs.iloc[i])
    
    
        dic = {"track_id":index,"y":augmented}
        df = pd.DataFrame(dic)
        
        df["bit_rate"] = gs.iloc[i].bit_rate
        df["all_genres"] = 0
        df["all_genres"] = [gs.iloc[i].all_genres] *len(df)
        df["sr"] = gs.iloc[i].sr
        
        big_test = pd.concat([big_test,df], ignore_index=True)
        
    return big_test

### Augment the Data!

In [17]:
enc = MultiLabelBinarizer()
encgenres = pd.DataFrame(enc.fit_transform(big_data["all_genres"]))
classes = []
for i in range(len(enc.classes_)):
    classes.append([i,len(encgenres.loc[encgenres[i]==1])])
    
for genre in sorted(classes, key=lambda x: x[1]):
    
    if l[1] < 500:
        augmented = []
        index = []
        df = augment_data(enc.classes_[genre[0]], 500)
        big_data = pd.concat([big_data, df], ignore_index=True)
        encgenres = pd.DataFrame(enc.fit_transform(big_data["all_genres"]))
        classes = []
        for i in range(len(enc.classes_)):
            classes.append([i,len(encgenres.loc[encgenres[i]==1])])
        

Indie-Rock: 0.218
Indie-Rock: 0.232
Indie-Rock: 0.26
Indie-Rock: 0.302
Indie-Rock: 0.358
Indie-Rock: 0.428
Indie-Rock: 0.512
Indie-Rock: 0.61
Indie-Rock: 0.722
Indie-Rock: 0.848
Indie-Rock: 0.988
International: 0.224
International: 0.238
International: 0.266
International: 0.308
International: 0.364
International: 0.434
International: 0.518
International: 0.616
International: 0.728
International: 0.854
International: 0.994
Experimental: 0.314
Experimental: 0.32
Experimental: 0.332
Experimental: 0.35
Experimental: 0.374
Experimental: 0.404
Experimental: 0.44
Experimental: 0.482
Experimental: 0.53
Experimental: 0.584
Experimental: 0.644
Experimental: 0.71
Experimental: 0.782
Experimental: 0.86
Experimental: 0.944
Electronic: 0.372
Electronic: 0.378
Electronic: 0.39
Electronic: 0.408
Electronic: 0.432
Electronic: 0.462
Electronic: 0.498
Electronic: 0.54
Electronic: 0.588
Electronic: 0.642
Electronic: 0.702
Electronic: 0.768
Electronic: 0.84
Electronic: 0.918
Hip-Hop: 0.39
Hip-Hop: 0.396
H

In [18]:
big_data

,track_id,bit_rate,all_genres,y,sr
0,OOO-000002,256000,[Hip-Hop],"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",16000
1,OOO-000003,256000,[Hip-Hop],"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",16000
2,OOO-000005,256000,[Hip-Hop],"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",16000
3,OOO-000134,256000,[Hip-Hop],"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",16000
4,OOO-000136,256000,[Rock],"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",16000
...,...,...,...,...,...
4170,034-000207,256000,[Folk],"[0.0039603878492535894, 0.005395981476878981, ...",16000
4171,035-000207,256000,[Folk],"[-0.015787465626385718, -0.0016790618280483176...",16000
4172,036-000208,256000,[Folk],"[-0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0....",16000
4173,037-000208,256000,[Folk],"[0.002946226188674887, 0.013041241494400349, 0...",16000


### Much better balance. Rock is still above the rest, but not by as much!

In [19]:
enc = MultiLabelBinarizer()
encgenres = pd.DataFrame(enc.fit_transform(big_data["all_genres"]))
classes = []

for i in range(len(enc.classes_)):
    classes.append([i,len(encgenres.loc[encgenres[i]==1])])

for l in sorted(classes, key=lambda x: x[1], reverse=True):
    print(f"{enc.classes_[l[0]]}: {l[1]}")

Rock: 1559
International: 574
Indie-Rock: 571
Punk: 529
Experimental: 517
Folk: 514
Hip-Hop: 510
Electronic: 501


### Time to extract info from the data!

In [891]:
hop_length = 1024
big_data["mfcc"] = big_data.apply(lambda row: librosa.feature.mfcc(y=row["y"], sr=row["sr"], hop_length=hop_length, n_mfcc=10), axis=1)

In [892]:
big_data["chroma"] = big_data.apply(lambda row: librosa.feature.chroma_stft(y=row["y"], sr=row["sr"],n_chroma=10, hop_length=hop_length,n_fft=2048), axis=1)

In [925]:
big_data["spectral"] = big_data.apply(lambda row: librosa.feature.spectral_centroid(y=row["y"], sr=row["sr"],hop_length=hop_length),axis=1)

In [959]:
big_data["zcross"] = big_data.apply(lambda row: librosa.feature.zero_crossing_rate(row["y"],hop_length=hop_length),axis=1)

In [ ]:
big_data

In [25]:
encoded = enc.fit_transform(big_data["all_genres"])
encgenres = pd.DataFrame(encoded, columns=enc.classes_)
big_data = pd.concat([big_data, encgenres], axis=1)

### Final dataframe in all it's glory!

In [1028]:
big_data

,track_id,bit_rate,all_genres,y,sr,mfcc,chroma,spectral,zcross,Electronic,Experimental,Folk,Hip-Hop,Indie-Rock,International,Punk,Rock
3797,012-000369,256000,"[Punk, Rock]","[-0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0....",16000,"[[-261.75668, -135.58673, -15.239572, -12.3259...","[[0.26851124, 0.04189693, 0.78985626, 0.161644...","[[1481.6807778740895, 1708.9312617305268, 1057...","[[0.01513671875, 0.0400390625, 0.0498046875, 0...",0,0,0,0,0,0,1,1
1781,003-000824,256000,"[Indie-Rock, Rock]","[-0.00035237466, -0.00041079277, -0.0004094517...",16000,"[[-77.59873, 50.99259, 51.84704, 30.608698, 36...","[[0.31460342, 0.45461187, 0.45708284, 0.434485...","[[1594.1823829111877, 1701.5023615218697, 1888...","[[0.068359375, 0.1494140625, 0.158203125, 0.12...",0,0,0,0,1,0,0,1
3922,002-000139,256000,[Folk],"[-0.0029076350251451165, -0.000255725846660055...",16000,"[[-232.60339620083997, -114.0510718783331, -10...","[[1.0, 0.03371930248921651, 0.0307432699198376...","[[3885.57570037974, 2373.8713084679302, 2922.8...","[[0.185546875, 0.25732421875, 0.1484375, 0.160...",0,0,1,0,0,0,0,0
2021,033-000830,256000,"[Indie-Rock, Rock]","[-2.7789813e-05, -9.041599e-05, 3.7759415e-05,...",16000,"[[-82.51235, 42.72989, 15.14737, 27.91318, 40....","[[0.88533676, 0.5499044, 0.19738185, 0.2592471...","[[1229.034387857203, 1430.1723981548362, 1511....","[[0.02490234375, 0.0693359375, 0.078125, 0.078...",0,0,0,0,1,0,0,1
3693,022-000896,160000,"[Punk, Rock]","[0.016127849910157782, -0.00218808204209632, -...",16000,"[[-106.94361725707908, 43.24532644570139, 61.8...","[[1.0, 1.0, 0.9583717596070912, 1.0, 1.0, 1.0,...","[[2679.8670802710053, 2087.4985474172277, 2298...","[[0.13916015625, 0.22998046875, 0.21044921875,...",0,0,0,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1825,005-000824,256000,"[Indie-Rock, Rock]","[-0.00035237466, -0.00041079277, -0.0004094517...",16000,"[[-77.59873, 50.99259, 51.84704, 30.608698, 36...","[[0.31460342, 0.45461187, 0.45708284, 0.434485...","[[1594.1823829111877, 1701.5023615218697, 1888...","[[0.068359375, 0.1494140625, 0.158203125, 0.12...",0,0,0,0,1,0,0,1
2096,038-003401,256000,"[Indie-Rock, Rock]","[-4.1383824e-05, 1.6528917e-05, 2.2222854e-05,...",16000,"[[-214.4505, -82.32879, -68.93448, -114.964745...","[[0.4040403, 0.05229712, 0.08758459, 0.1518849...","[[1995.9936154387156, 2064.2353231500338, 2171...","[[0.0625, 0.126953125, 0.1611328125, 0.1933593...",0,0,0,0,1,0,0,1
2705,000-000148,256000,[Experimental],"[-0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0....",16000,"[[-148.78394, -2.6720407, 4.8523464, 22.647324...","[[0.18146876, 0.23321348, 0.10517499, 0.095677...","[[921.5792350792062, 1081.1882811801433, 1180....","[[0.02197265625, 0.0673828125, 0.0869140625, 0...",0,1,0,0,0,0,0,0
2510,060-000853,256000,[International],"[-0.011478485780449998, -0.01089870431992549, ...",16000,"[[-143.55179495411434, -65.95086143357136, -72...","[[0.3280126473277786, 0.11603360777912516, 0.2...","[[2330.5506054735656, 1756.303577730089, 1962....","[[0.115234375, 0.15673828125, 0.08642578125, 0...",0,0,0,0,0,1,0,0


### Get it ready for machine learning

In [109]:
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
import tensorflow as tf

In [29]:
big_data = big_data.sample(frac=1)

In [975]:
Y = big_data[["Electronic", "Experimental", "Folk", "Hip-Hop", "International", "Punk", "Rock"]]
X = big_data[["bit_rate", "y", "mfcc", "chroma", "spectral"]]
X.loc[:,'mfcc'] = X['mfcc'].apply(lambda x: x.reshape(-1,10,469,1))
mfcc = np.array(X['mfcc'].tolist())
trainX, testX, trainY, testY = train_test_split(mfcc, Y, stratify=Y, test_size=0.2)

In [532]:
mfcc.shape

(4175, 1, 10, 469, 1)

### Dummy Methods

In [53]:
dummy_clf = DummyClassifier(strategy="uniform")
dummy_clf.fit(trainX, trainY)
preds = dummy_clf.predict(testX)

print(f1_score(testY, preds, average="macro"))
print(precision_score(testY, preds, average="macro"))
print(recall_score(testY, preds, average="macro"))
print(classification_report(testY, preds))

0.22954584367956749
0.15923486239750526
0.4929623209236055
              precision    recall  f1-score   support

           0       0.13      0.54      0.21       100
           1       0.11      0.45      0.18       103
           2       0.13      0.54      0.21       103
           3       0.13      0.50      0.20       102
           4       0.14      0.52      0.22       115
           5       0.11      0.43      0.18       106
           6       0.37      0.46      0.41       312

   micro avg       0.16      0.49      0.24       941
   macro avg       0.16      0.49      0.23       941
weighted avg       0.20      0.49      0.27       941
 samples avg       0.15      0.49      0.23       941



C:\Users\miles\school\cs5850\Genre Classification\project\env\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [54]:
dummy_clf = DummyClassifier(strategy="stratified")
dummy_clf.fit(trainX, trainY)
preds = dummy_clf.predict(testX)

print(f1_score(testY, preds, average="macro"))
print(precision_score(testY, preds, average="macro"))
print(recall_score(testY, preds, average="macro"))
print(classification_report(testY, preds))

0.182679703403005
0.18281176272157396
0.18321875544594407
              precision    recall  f1-score   support

           0       0.16      0.14      0.15       100
           1       0.16      0.17      0.17       103
           2       0.11      0.12      0.11       103
           3       0.14      0.16      0.15       102
           4       0.08      0.10      0.09       115
           5       0.21      0.20      0.20       106
           6       0.41      0.40      0.40       312

   micro avg       0.23      0.23      0.23       941
   macro avg       0.18      0.18      0.18       941
weighted avg       0.23      0.23      0.23       941
 samples avg       0.17      0.22      0.18       941



C:\Users\miles\school\cs5850\Genre Classification\project\env\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Comparing Loss Functions

In [976]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(1, 10, 469, 1)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(7)
])
model.compile(loss="CategoricalCrossentropy", optimizer="adam", metrics=['accuracy'])
model.fit(trainX, trainY, validation_data=(testX, testY), epochs=10, batch_size=64)
eval_result = model.evaluate(testX, testY)
print("[test loss, test accuracy]:", eval_result)

Epoch 1/10
53/53 [==============================] - 1s 11ms/step - loss: 9.3140 - accuracy: 0.1308 - val_loss: 9.5763 - val_accuracy: 0.1329
Epoch 2/10
53/53 [==============================] - 0s 6ms/step - loss: 10.8180 - accuracy: 0.1368 - val_loss: 11.5260 - val_accuracy: 0.1581
Epoch 3/10
53/53 [==============================] - 0s 5ms/step - loss: 9.6560 - accuracy: 0.1524 - val_loss: 8.5146 - val_accuracy: 0.1689
Epoch 4/10
53/53 [==============================] - 0s 5ms/step - loss: 8.9135 - accuracy: 0.1530 - val_loss: 10.5009 - val_accuracy: 0.1892
Epoch 5/10
53/53 [==============================] - 0s 5ms/step - loss: 9.9134 - accuracy: 0.1632 - val_loss: 9.3813 - val_accuracy: 0.1868
Epoch 6/10
53/53 [==============================] - 0s 4ms/step - loss: 9.6420 - accuracy: 0.1581 - val_loss: 10.7906 - val_accuracy: 0.1880
Epoch 7/10
53/53 [==============================] - 0s 4ms/step - loss: 10.0335 - accuracy: 0.1719 - val_loss: 10.4044 - val_accuracy: 0.1832
Epoch 8/10
53

In [979]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(1, 10, 469, 1)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(7)
])
model.compile(loss="CategoricalHinge", optimizer="adam", metrics=['accuracy'])
model.fit(trainX, trainY, validation_data=(testX, testY), epochs=10, batch_size=64)
eval_result = model.evaluate(testX, testY)
print("[test loss, test accuracy]:", eval_result)

Epoch 1/10
53/53 [==============================] - 1s 10ms/step - loss: 33.9662 - accuracy: 0.3716 - val_loss: 2.1518 - val_accuracy: 0.3198
Epoch 2/10
53/53 [==============================] - 0s 3ms/step - loss: 1.2159 - accuracy: 0.3362 - val_loss: 2.2842 - val_accuracy: 0.3138
Epoch 3/10
53/53 [==============================] - 0s 3ms/step - loss: 1.1838 - accuracy: 0.3446 - val_loss: 2.3453 - val_accuracy: 0.3377
Epoch 4/10
53/53 [==============================] - 0s 3ms/step - loss: 1.0098 - accuracy: 0.3521 - val_loss: 1.9797 - val_accuracy: 0.3808
Epoch 5/10
53/53 [==============================] - 0s 3ms/step - loss: 0.8682 - accuracy: 0.3557 - val_loss: 3.7068 - val_accuracy: 0.3365
Epoch 6/10
53/53 [==============================] - 0s 3ms/step - loss: 0.8783 - accuracy: 0.3719 - val_loss: 2.0879 - val_accuracy: 0.3749
Epoch 7/10
53/53 [==============================] - 0s 4ms/step - loss: 0.7535 - accuracy: 0.3647 - val_loss: 2.1062 - val_accuracy: 0.3425
Epoch 8/10
53/53 [

In [977]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(1, 10, 469, 1)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(7)
])
model.compile(loss="Hinge", optimizer="adam", metrics=['accuracy'])
model.fit(trainX, trainY, validation_data=(testX, testY), epochs=10, batch_size=64)
eval_result = model.evaluate(testX, testY)
print("[test loss, test accuracy]:", eval_result)

Epoch 1/10
53/53 [==============================] - 1s 10ms/step - loss: 6.0048 - accuracy: 0.4006 - val_loss: 0.9793 - val_accuracy: 0.3820
Epoch 2/10
53/53 [==============================] - 0s 7ms/step - loss: 0.8316 - accuracy: 0.3868 - val_loss: 0.8557 - val_accuracy: 0.4132
Epoch 3/10
53/53 [==============================] - 0s 5ms/step - loss: 0.7354 - accuracy: 0.4141 - val_loss: 0.7991 - val_accuracy: 0.4335
Epoch 4/10
53/53 [==============================] - 0s 5ms/step - loss: 0.6237 - accuracy: 0.4491 - val_loss: 0.6860 - val_accuracy: 0.4683
Epoch 5/10
53/53 [==============================] - 0s 4ms/step - loss: 0.5341 - accuracy: 0.4614 - val_loss: 0.6682 - val_accuracy: 0.4982
Epoch 6/10
53/53 [==============================] - 0s 4ms/step - loss: 0.5010 - accuracy: 0.4554 - val_loss: 0.5397 - val_accuracy: 0.4766
Epoch 7/10
53/53 [==============================] - 0s 4ms/step - loss: 0.4453 - accuracy: 0.4790 - val_loss: 0.5627 - val_accuracy: 0.4719
Epoch 8/10
53/53 [=

In [978]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(1, 10, 469, 1)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(7)
])
model.compile(loss="SquaredHinge", optimizer="adam", metrics=['accuracy'])
model.fit(trainX, trainY, validation_data=(testX, testY), epochs=10, batch_size=64)

eval_result = model.evaluate(testX, testY)
print("[test loss, test accuracy]:", eval_result)

Epoch 1/10
53/53 [==============================] - 1s 10ms/step - loss: 319.1454 - accuracy: 0.2892 - val_loss: 14.7599 - val_accuracy: 0.3006
Epoch 2/10
53/53 [==============================] - 0s 3ms/step - loss: 7.4226 - accuracy: 0.3045 - val_loss: 23.4602 - val_accuracy: 0.3377
Epoch 3/10
53/53 [==============================] - 0s 3ms/step - loss: 8.9615 - accuracy: 0.3171 - val_loss: 17.9047 - val_accuracy: 0.3246
Epoch 4/10
53/53 [==============================] - 0s 3ms/step - loss: 3.7673 - accuracy: 0.3219 - val_loss: 19.4199 - val_accuracy: 0.3281
Epoch 5/10
53/53 [==============================] - 0s 3ms/step - loss: 1.7914 - accuracy: 0.3380 - val_loss: 12.2841 - val_accuracy: 0.3293
Epoch 6/10
53/53 [==============================] - 0s 3ms/step - loss: 1.5304 - accuracy: 0.3410 - val_loss: 23.7178 - val_accuracy: 0.3257
Epoch 7/10
53/53 [==============================] - 0s 3ms/step - loss: 1.1549 - accuracy: 0.3374 - val_loss: 16.4001 - val_accuracy: 0.3377
Epoch 8/10

### Tuned FCNN

In [998]:
def model_builder(hp):
    model = keras.Sequential()
    model.add(tf.keras.layers.Flatten(input_shape=(1, 10, 469, 1)))
    
    hp_units = hp.Int('units', min_value=32, max_value=128, step=32)
    hp_units1 = hp.Int('units1', min_value=64, max_value=256, step=64)
    hp_units2 = hp.Int('units2', min_value=64, max_value=256, step=64)
    hp_units3 = hp.Int('units3', min_value=64, max_value=256, step=64)
    hp_units4 = hp.Int('units4', min_value=64, max_value=256, step=64)
    
    model.add(keras.layers.Dense(units=hp_units, activation='relu'))
    model.add(keras.layers.Dense(units=hp_units1, activation='relu'))
    model.add(keras.layers.Dense(units=hp_units2, activation='relu'))
    model.add(keras.layers.Dense(units=hp_units3, activation='relu'))
    model.add(keras.layers.Dense(units=hp_units4, activation='relu'))
    
    hp_drop = hp.Float('drop', min_value=0, max_value=0.5, step=0.1)
    
    model.add(tf.keras.layers.Dropout(rate=hp_drop),)
    
    model.add(keras.layers.Dense(7))

    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss=keras.losses.CategoricalHinge(),
                metrics=['accuracy'])

    return model

In [1003]:
tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3)
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
tuner.search(trainX, trainY, epochs=100, validation_split=0.2, callbacks=[stop_early])
fcbest_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

INFO:tensorflow:Reloading Tuner from .\untitled_project\tuner0.json
INFO:tensorflow:Oracle triggered exit


In [1000]:
model = tuner.hypermodel.build(best_hps)
history = model.fit(trainX, trainY, epochs=100, validation_split=0.2)

val_acc = history.history['val_accuracy']
best_epoch = val_acc.index(max(val_acc))+1
print(best_epoch)

Epoch 1/100
84/84 [==============================] - 1s 9ms/step - loss: 1.0232 - accuracy: 0.2975 - val_loss: 0.9702 - val_accuracy: 0.4207
Epoch 2/100
84/84 [==============================] - 0s 5ms/step - loss: 0.9275 - accuracy: 0.4615 - val_loss: 0.8072 - val_accuracy: 0.5479
Epoch 3/100
84/84 [==============================] - 0s 5ms/step - loss: 0.7574 - accuracy: 0.5696 - val_loss: 0.6075 - val_accuracy: 0.6198
Epoch 4/100
84/84 [==============================] - 0s 5ms/step - loss: 0.5540 - accuracy: 0.6490 - val_loss: 0.4772 - val_accuracy: 0.6407
Epoch 5/100
84/84 [==============================] - 0s 5ms/step - loss: 0.4527 - accuracy: 0.6677 - val_loss: 0.4552 - val_accuracy: 0.6392
Epoch 6/100
84/84 [==============================] - 0s 4ms/step - loss: 0.3965 - accuracy: 0.6972 - val_loss: 0.4324 - val_accuracy: 0.6512
Epoch 7/100
84/84 [==============================] - 0s 4ms/step - loss: 0.3573 - accuracy: 0.7107 - val_loss: 0.4180 - val_accuracy: 0.6632
Epoch 8/100
8

Epoch 59/100
84/84 [==============================] - 0s 5ms/step - loss: 0.0134 - accuracy: 0.8743 - val_loss: 0.7266 - val_accuracy: 0.6497
Epoch 60/100
84/84 [==============================] - 0s 5ms/step - loss: 0.0169 - accuracy: 0.8724 - val_loss: 0.8465 - val_accuracy: 0.6093
Epoch 61/100
84/84 [==============================] - 0s 5ms/step - loss: 0.0166 - accuracy: 0.8713 - val_loss: 0.7032 - val_accuracy: 0.6467
Epoch 62/100
84/84 [==============================] - 0s 5ms/step - loss: 0.0102 - accuracy: 0.8743 - val_loss: 0.6690 - val_accuracy: 0.6437
Epoch 63/100
84/84 [==============================] - 0s 5ms/step - loss: 0.0017 - accuracy: 0.8769 - val_loss: 0.6993 - val_accuracy: 0.6392
Epoch 64/100
84/84 [==============================] - 0s 5ms/step - loss: 2.1103e-04 - accuracy: 0.8769 - val_loss: 0.7177 - val_accuracy: 0.6722
Epoch 65/100
84/84 [==============================] - 0s 5ms/step - loss: 0.0037 - accuracy: 0.8754 - val_loss: 0.7785 - val_accuracy: 0.6332
Ep

In [1001]:
hypermodel = tuner.hypermodel.build(best_hps)

hypermodel.fit(trainX, trainY, epochs=best_epoch, validation_split=0.2)

Epoch 1/23
84/84 [==============================] - 1s 6ms/step - loss: 1.0221 - accuracy: 0.2788 - val_loss: 0.9721 - val_accuracy: 0.4895
Epoch 2/23
84/84 [==============================] - 0s 5ms/step - loss: 0.9454 - accuracy: 0.4618 - val_loss: 0.8216 - val_accuracy: 0.5165
Epoch 3/23
84/84 [==============================] - 0s 5ms/step - loss: 0.8010 - accuracy: 0.5284 - val_loss: 0.6805 - val_accuracy: 0.5719
Epoch 4/23
84/84 [==============================] - 0s 5ms/step - loss: 0.6460 - accuracy: 0.6063 - val_loss: 0.5660 - val_accuracy: 0.6243
Epoch 5/23
84/84 [==============================] - 0s 5ms/step - loss: 0.5289 - accuracy: 0.6497 - val_loss: 0.5245 - val_accuracy: 0.6228
Epoch 6/23
84/84 [==============================] - 0s 5ms/step - loss: 0.4256 - accuracy: 0.6875 - val_loss: 0.4701 - val_accuracy: 0.6467
Epoch 7/23
84/84 [==============================] - 0s 5ms/step - loss: 0.3839 - accuracy: 0.7013 - val_loss: 0.5096 - val_accuracy: 0.6198
Epoch 8/23
84/84 [==

In [1002]:
eval_result = hypermodel.evaluate(testX, testY)
print("[test loss, test accuracy]:", eval_result)

27/27 [==============================] - 0s 2ms/step - loss: 0.4841 - accuracy: 0.6683
[test loss, test accuracy]: [0.48412856459617615, 0.6682634949684143]


### Prepare for LSTM

In [268]:
import keras_tuner as kt
from tensorflow.keras import layers
import keras
from tensorflow import keras
from tensorflow.keras import layers

In [313]:
Y = big_data[["Electronic", "Experimental", "Folk", "Hip-Hop", "International", "Punk", "Rock"]]
X = big_data[["mfcc", "chroma", "spectral"]]
mfcc = np.array(X['mfcc'].values.tolist())
steps = mfcc.shape[1]
features = mfcc.shape[2]
mfcc = mfcc.reshape((-1,steps,features))
trainX, testX, trainY, testY = train_test_split(mfcc, Y, stratify=Y, test_size=0.2)

### Tuned LSTM

In [255]:
def model_builder(hp):
    model = keras.Sequential()
    model.add(keras.layers.Input(shape=(10, 469)))
    
    hp_units = hp.Int('units', min_value=50, max_value=250, step=50)
    hp_drop = hp.Float('drop', min_value=0, max_value=0.5, step=0.1)
    model.add(keras.layers.Bidirectional(keras.layers.LSTM(units=hp_units, dropout=hp_drop)))
    
    model.add(keras.layers.Dense(416, activation='relu'))
    hp_units1 = hp.Int('units1', min_value=50, max_value=250, step=50)
    model.add(keras.layers.Dense(units=hp_units))
    model.add(keras.layers.Dense(7))

    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss=keras.losses.CategoricalHinge(),
                metrics=['accuracy'])

    return model

In [257]:
tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3)
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

In [258]:
tuner.search(trainX, trainY, epochs=50, validation_split=0.2, callbacks=[stop_early])

Trial 30 Complete [00h 00m 09s]
val_accuracy: 0.667664647102356

Best val_accuracy So Far: 0.67514967918396
Total elapsed time: 00h 03m 01s
INFO:tensorflow:Oracle triggered exit


In [259]:
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

In [260]:
model = tuner.hypermodel.build(best_hps)
history = model.fit(trainX, trainY, epochs=50, validation_split=0.2)

val_acc = history.history['val_accuracy']
best_epoch = val_acc.index(max(val_acc))+1
print(best_epoch)

Epoch 1/50
84/84 [==============================] - 3s 14ms/step - loss: 0.7758 - accuracy: 0.5449 - val_loss: 0.5769 - val_accuracy: 0.6228
Epoch 2/50
84/84 [==============================] - 1s 6ms/step - loss: 0.3927 - accuracy: 0.7100 - val_loss: 0.4987 - val_accuracy: 0.6392
Epoch 3/50
84/84 [==============================] - 1s 6ms/step - loss: 0.2966 - accuracy: 0.7534 - val_loss: 0.4723 - val_accuracy: 0.6482
Epoch 4/50
84/84 [==============================] - 1s 6ms/step - loss: 0.2213 - accuracy: 0.7908 - val_loss: 0.4786 - val_accuracy: 0.6452
Epoch 5/50
84/84 [==============================] - 1s 6ms/step - loss: 0.1703 - accuracy: 0.8132 - val_loss: 0.4770 - val_accuracy: 0.6437
Epoch 6/50
84/84 [==============================] - 1s 6ms/step - loss: 0.1223 - accuracy: 0.8376 - val_loss: 0.4795 - val_accuracy: 0.6467
Epoch 7/50
84/84 [==============================] - 1s 6ms/step - loss: 0.0734 - accuracy: 0.8540 - val_loss: 0.4920 - val_accuracy: 0.6422
Epoch 8/50
84/84 [=

In [261]:
hypermodel = tuner.hypermodel.build(best_hps)

hypermodel.fit(trainX, trainY, epochs=best_epoch, validation_split=0.2)

Epoch 1/29
84/84 [==============================] - 3s 16ms/step - loss: 0.8021 - accuracy: 0.5153 - val_loss: 0.5854 - val_accuracy: 0.6272
Epoch 2/29
84/84 [==============================] - 1s 6ms/step - loss: 0.3962 - accuracy: 0.7141 - val_loss: 0.5029 - val_accuracy: 0.6452
Epoch 3/29
84/84 [==============================] - 1s 6ms/step - loss: 0.2954 - accuracy: 0.7552 - val_loss: 0.4952 - val_accuracy: 0.6392
Epoch 4/29
84/84 [==============================] - 1s 6ms/step - loss: 0.2304 - accuracy: 0.7934 - val_loss: 0.4865 - val_accuracy: 0.6422
Epoch 5/29
84/84 [==============================] - 1s 6ms/step - loss: 0.1731 - accuracy: 0.8185 - val_loss: 0.4861 - val_accuracy: 0.6632
Epoch 6/29
84/84 [==============================] - 1s 7ms/step - loss: 0.1255 - accuracy: 0.8357 - val_loss: 0.4904 - val_accuracy: 0.6482
Epoch 7/29
84/84 [==============================] - 1s 7ms/step - loss: 0.0844 - accuracy: 0.8507 - val_loss: 0.4887 - val_accuracy: 0.6512
Epoch 8/29
84/84 [=

In [262]:
eval_result = hypermodel.evaluate(testX, testY)
print("[test loss, test accuracy]:", eval_result)

27/27 [==============================] - 0s 10ms/step - loss: 0.5300 - accuracy: 0.6695
[test loss, test accuracy]: [0.5299615263938904, 0.6694610714912415]


### Baseline LSTMs

In [238]:
model = tf.keras.models.Sequential([
  tf.keras.Input(shape=(10, 469)),
  tf.keras.layers.LSTM(100, dropout=0.2),
  tf.keras.layers.Dense(7)
])
model.compile(loss="CategoricalHinge", optimizer="adam", metrics=['accuracy'])
model.fit(trainX, trainY, validation_data=(testX, testY), epochs=10, batch_size=64)
eval_result = model.evaluate(testX, testY)
print("[test loss, test accuracy]:", eval_result)

Epoch 1/10
53/53 [==============================] - 2s 16ms/step - loss: 1.0069 - accuracy: 0.4425 - val_loss: 0.5838 - val_accuracy: 0.6240
Epoch 2/10
53/53 [==============================] - 0s 7ms/step - loss: 0.6000 - accuracy: 0.6344 - val_loss: 0.5196 - val_accuracy: 0.6539
Epoch 3/10
53/53 [==============================] - 0s 5ms/step - loss: 0.5000 - accuracy: 0.6680 - val_loss: 0.5146 - val_accuracy: 0.6407
Epoch 4/10
53/53 [==============================] - 0s 5ms/step - loss: 0.4341 - accuracy: 0.6901 - val_loss: 0.4880 - val_accuracy: 0.6551
Epoch 5/10
53/53 [==============================] - 0s 5ms/step - loss: 0.3935 - accuracy: 0.7057 - val_loss: 0.4796 - val_accuracy: 0.6695
Epoch 6/10
53/53 [==============================] - 0s 5ms/step - loss: 0.3604 - accuracy: 0.7266 - val_loss: 0.4772 - val_accuracy: 0.6671
Epoch 7/10
53/53 [==============================] - 0s 5ms/step - loss: 0.3127 - accuracy: 0.7443 - val_loss: 0.4823 - val_accuracy: 0.6563
Epoch 8/10
53/53 [=

In [237]:
model = tf.keras.models.Sequential([
  tf.keras.Input(shape=(10, 469)),
  tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(100, dropout=0.2)),
  tf.keras.layers.Dense(7)
])
model.compile(loss="CategoricalHinge", optimizer="adam", metrics=['accuracy'])
model.fit(trainX, trainY, validation_data=(testX, testY), epochs=10, batch_size=64)
eval_result = model.evaluate(testX, testY)
print("[test loss, test accuracy]:", eval_result)

Epoch 1/10
53/53 [==============================] - 3s 21ms/step - loss: 0.9546 - accuracy: 0.4674 - val_loss: 0.5447 - val_accuracy: 0.6371
Epoch 2/10
53/53 [==============================] - 0s 7ms/step - loss: 0.5534 - accuracy: 0.6569 - val_loss: 0.4827 - val_accuracy: 0.6527
Epoch 3/10
53/53 [==============================] - 0s 7ms/step - loss: 0.4475 - accuracy: 0.6883 - val_loss: 0.4702 - val_accuracy: 0.6635
Epoch 4/10
53/53 [==============================] - 0s 7ms/step - loss: 0.3993 - accuracy: 0.7018 - val_loss: 0.4634 - val_accuracy: 0.6754
Epoch 5/10
53/53 [==============================] - 0s 7ms/step - loss: 0.3611 - accuracy: 0.7228 - val_loss: 0.4664 - val_accuracy: 0.6599
Epoch 6/10
53/53 [==============================] - 0s 7ms/step - loss: 0.3103 - accuracy: 0.7509 - val_loss: 0.4670 - val_accuracy: 0.6611
Epoch 7/10
53/53 [==============================] - 0s 7ms/step - loss: 0.2915 - accuracy: 0.7509 - val_loss: 0.4600 - val_accuracy: 0.6707
Epoch 8/10
53/53 [=

### Tuned Transformer

In [263]:
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    # Normalization and Attention
    x = layers.LayerNormalization(epsilon=1e-6)(inputs)
    x = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout
    )(x, x)
    x = layers.Dropout(dropout)(x)
    res = x + inputs

    # Feed Forward Part
    x = layers.LayerNormalization(epsilon=1e-6)(res)
    x = layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(x)
    x = layers.Dropout(dropout)(x)
    x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
    return x + res

In [269]:
def build_model(
    input_shape,
    head_size,
    num_heads,
    ff_dim,
    num_transformer_blocks,
    mlp_units,
    dropout=0,
    mlp_dropout=0,
):
    inputs = keras.Input(shape=input_shape)
    x = inputs
    for _ in range(num_transformer_blocks):
        x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)

    x = layers.GlobalAveragePooling1D(data_format="channels_first")(x)
    for dim in mlp_units:
        x = layers.Dense(dim, activation="relu")(x)
        x = layers.Dropout(mlp_dropout)(x)
    outputs = layers.Dense(7, activation="softmax")(x)
    return keras.Model(inputs, outputs)

In [273]:
input_shape = trainX.shape[1:]

model.summary()

callbacks = [keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)]



Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_9 (InputLayer)           [(None, 10, 469)]    0           []                               
                                                                                                  
 layer_normalization_32 (LayerN  (None, 10, 469)     938         ['input_9[0][0]']                
 ormalization)                                                                                    
                                                                                                  
 multi_head_attention_16 (Multi  (None, 10, 469)     1924565     ['layer_normalization_32[0][0]', 
 HeadAttention)                                                   'layer_normalization_32[0][0]'] 
                                                                                            

 ambda)                                                           'tf.__operators__.add_37[0][0]']
                                                                                                  
 layer_normalization_39 (LayerN  (None, 10, 469)     938         ['tf.__operators__.add_38[0][0]']
 ormalization)                                                                                    
                                                                                                  
 conv1d_38 (Conv1D)             (None, 10, 4)        1880        ['layer_normalization_39[0][0]'] 
                                                                                                  
 dropout_43 (Dropout)           (None, 10, 4)        0           ['conv1d_38[0][0]']              
                                                                                                  
 conv1d_39 (Conv1D)             (None, 10, 469)      2345        ['dropout_43[0][0]']             
          

27/27 [==============================] - 0s 6ms/step - loss: 1.1008 - accuracy: 0.3437


[1.1007895469665527, 0.34371256828308105]

In [288]:

def model_builder(hp):
    
    hp_blocks = hp.Int("blocks", min_value=4, max_value=5, step=2)
    hp_units = hp.Int('units', min_value=64, max_value=256, step=64)
    hp_drop = hp.Float('drop', min_value=0.1, max_value=0.3, step=0.1)
    hp_drop1 = hp.Float('drop1', min_value=0.1, max_value=0.3, step=0.1)
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
                      
    model = build_model(
    input_shape,
    head_size=256,
    num_heads=4,
    ff_dim=4,
    num_transformer_blocks=hp_blocks,
    mlp_units=[hp_units],
    mlp_dropout=hp_drop,
    dropout=hp_drop1,
)
    model.compile(
    loss=keras.losses.CategoricalHinge(),
    optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
    metrics=["accuracy"],
)
    return model

In [289]:
tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=20,
                     factor=3)
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

In [290]:
tuner.search(trainX, trainY, epochs=20, validation_split=0.2, callbacks=[stop_early])

Trial 30 Complete [00h 00m 40s]
val_accuracy: 0.26047903299331665

Best val_accuracy So Far: 0.5883233547210693
Total elapsed time: 00h 08m 32s
INFO:tensorflow:Oracle triggered exit


In [291]:
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

In [292]:
model = tuner.hypermodel.build(best_hps)
history = model.fit(trainX, trainY, epochs=50, validation_split=0.2)

val_acc = history.history['val_accuracy']
best_epoch = val_acc.index(max(val_acc))+1
print(best_epoch)

Epoch 1/50
84/84 [==============================] - 5s 26ms/step - loss: 1.5476 - accuracy: 0.1557 - val_loss: 1.5222 - val_accuracy: 0.1632
Epoch 2/50
84/84 [==============================] - 2s 20ms/step - loss: 1.4415 - accuracy: 0.1976 - val_loss: 1.2599 - val_accuracy: 0.2620
Epoch 3/50
84/84 [==============================] - 2s 20ms/step - loss: 1.2937 - accuracy: 0.2788 - val_loss: 1.1580 - val_accuracy: 0.3278
Epoch 4/50
84/84 [==============================] - 2s 20ms/step - loss: 1.2073 - accuracy: 0.3185 - val_loss: 1.0941 - val_accuracy: 0.3623
Epoch 5/50
84/84 [==============================] - 2s 20ms/step - loss: 1.0491 - accuracy: 0.3896 - val_loss: 0.9245 - val_accuracy: 0.4311
Epoch 6/50
84/84 [==============================] - 2s 20ms/step - loss: 0.9159 - accuracy: 0.4386 - val_loss: 0.8286 - val_accuracy: 0.4716
Epoch 7/50
84/84 [==============================] - 2s 20ms/step - loss: 0.8627 - accuracy: 0.4581 - val_loss: 0.8126 - val_accuracy: 0.4731
Epoch 8/50
84

In [293]:
hypermodel = tuner.hypermodel.build(best_hps)

hypermodel.fit(trainX, trainY, epochs=best_epoch, validation_split=0.2)

Epoch 1/26
84/84 [==============================] - 5s 27ms/step - loss: 1.6536 - accuracy: 0.1340 - val_loss: 1.6092 - val_accuracy: 0.1392
Epoch 2/26
84/84 [==============================] - 2s 20ms/step - loss: 1.5766 - accuracy: 0.1707 - val_loss: 1.4654 - val_accuracy: 0.2260
Epoch 3/26
84/84 [==============================] - 2s 20ms/step - loss: 1.4707 - accuracy: 0.2350 - val_loss: 1.3905 - val_accuracy: 0.2650
Epoch 4/26
84/84 [==============================] - 2s 20ms/step - loss: 1.3080 - accuracy: 0.3132 - val_loss: 1.1638 - val_accuracy: 0.3578
Epoch 5/26
84/84 [==============================] - 2s 20ms/step - loss: 1.1237 - accuracy: 0.3855 - val_loss: 1.0434 - val_accuracy: 0.4222
Epoch 6/26
84/84 [==============================] - 2s 20ms/step - loss: 1.0438 - accuracy: 0.4154 - val_loss: 1.0143 - val_accuracy: 0.4237
Epoch 7/26
84/84 [==============================] - 2s 20ms/step - loss: 0.9945 - accuracy: 0.4274 - val_loss: 0.9231 - val_accuracy: 0.4461
Epoch 8/26
84

In [294]:
eval_result = hypermodel.evaluate(testX, testY)
print("[test loss, test accuracy]:", eval_result)

27/27 [==============================] - 0s 6ms/step - loss: 0.7496 - accuracy: 0.5090
[test loss, test accuracy]: [0.7496476173400879, 0.5089820623397827]


### Tuned Simple RNN

In [331]:
Y = big_data[["Electronic", "Experimental", "Folk", "Hip-Hop", "International", "Punk", "Rock"]]
X = big_data[["mfcc", "chroma", "spectral"]]
mfcc = np.array(X['mfcc'].values.tolist())
samples = mfcc.shape[0]
steps = mfcc.shape[2]
coef = 10
mfcc = mfcc[:,:steps*coef].reshape(samples,steps,coef)
trainX, testX, trainY, testY = train_test_split(mfcc, Y, stratify=Y, test_size=0.2)

In [338]:
def model_builder(hp):
    model = keras.Sequential()
    model.add(tf.keras.layers.Input(shape=(469, 10)))
    
    hp_units = hp.Int('units', min_value=32, max_value=128, step=32)
    hp_drop = hp.Float('drop', min_value=0, max_value=0.5, step=0.1)
    
    model.add(keras.layers.SimpleRNN(units=hp_units, dropout=hp_drop))

    model.add(keras.layers.Dense(7))

    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss=keras.losses.CategoricalHinge(),
                metrics=['accuracy'])

    return model

In [340]:
tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3)
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
tuner.search(trainX, trainY, epochs=20, validation_split=0.2, callbacks=[stop_early])
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

Trial 30 Complete [00h 04m 01s]
val_accuracy: 0.30239519476890564

Best val_accuracy So Far: 0.636227548122406
Total elapsed time: 00h 51m 35s
INFO:tensorflow:Oracle triggered exit


In [341]:
model = tuner.hypermodel.build(best_hps)
history = model.fit(trainX, trainY, epochs=20, validation_split=0.2)

val_acc = history.history['val_accuracy']
best_epoch = val_acc.index(max(val_acc))+1
print(best_epoch)

Epoch 1/20
84/84 [==============================] - 25s 290ms/step - loss: 1.2922 - accuracy: 0.3986 - val_loss: 0.8257 - val_accuracy: 0.5808
Epoch 2/20
84/84 [==============================] - 24s 287ms/step - loss: 0.6702 - accuracy: 0.6213 - val_loss: 0.6461 - val_accuracy: 0.6093
Epoch 3/20
84/84 [==============================] - 24s 288ms/step - loss: 0.5242 - accuracy: 0.6662 - val_loss: 0.6068 - val_accuracy: 0.6033
Epoch 4/20
84/84 [==============================] - 24s 288ms/step - loss: 0.4624 - accuracy: 0.6864 - val_loss: 0.6012 - val_accuracy: 0.6048
Epoch 5/20
84/84 [==============================] - 24s 287ms/step - loss: 0.3978 - accuracy: 0.7227 - val_loss: 0.5551 - val_accuracy: 0.6257
Epoch 6/20
84/84 [==============================] - 24s 284ms/step - loss: 0.3524 - accuracy: 0.7373 - val_loss: 0.5639 - val_accuracy: 0.6287
Epoch 7/20
84/84 [==============================] - 24s 283ms/step - loss: 0.3250 - accuracy: 0.7541 - val_loss: 0.5444 - val_accuracy: 0.6497

In [342]:
hypermodel = tuner.hypermodel.build(best_hps)

hypermodel.fit(trainX, trainY, epochs=best_epoch, validation_split=0.2)

Epoch 1/20
84/84 [==============================] - 25s 290ms/step - loss: 1.2571 - accuracy: 0.4210 - val_loss: 0.8462 - val_accuracy: 0.5659
Epoch 2/20
84/84 [==============================] - 24s 287ms/step - loss: 0.6824 - accuracy: 0.6299 - val_loss: 0.6772 - val_accuracy: 0.6213
Epoch 3/20
84/84 [==============================] - 24s 288ms/step - loss: 0.5395 - accuracy: 0.6755 - val_loss: 0.6329 - val_accuracy: 0.6078
Epoch 4/20
84/84 [==============================] - 24s 284ms/step - loss: 0.4691 - accuracy: 0.6995 - val_loss: 0.6099 - val_accuracy: 0.6228
Epoch 5/20
84/84 [==============================] - 24s 283ms/step - loss: 0.4020 - accuracy: 0.7242 - val_loss: 0.5740 - val_accuracy: 0.6422
Epoch 6/20
84/84 [==============================] - 24s 284ms/step - loss: 0.3592 - accuracy: 0.7429 - val_loss: 0.5690 - val_accuracy: 0.6422
Epoch 7/20
84/84 [==============================] - 24s 287ms/step - loss: 0.3299 - accuracy: 0.7534 - val_loss: 0.5574 - val_accuracy: 0.6422

In [343]:
eval_result = hypermodel.evaluate(testX, testY)
print("[test loss, test accuracy]:", eval_result)

27/27 [==============================] - 2s 57ms/step - loss: 0.6186 - accuracy: 0.6311
[test loss, test accuracy]: [0.6186283826828003, 0.6311377286911011]


### Tuned CNN

In [395]:
Y = big_data[["Electronic", "Experimental", "Folk", "Hip-Hop", "International", "Punk", "Rock"]]
X = big_data[["mfcc", "chroma", "spectral"]]
mfcc = np.array(X['mfcc'].values.tolist())
mfcc = np.reshape(mfcc, (mfcc.shape[0], mfcc.shape[1], mfcc.shape[2], 1))
trainX, testX, trainY, testY = train_test_split(mfcc, Y, stratify=Y, test_size=0.2)

In [452]:
def model_builder(hp):
    model = keras.Sequential()
    model.add(tf.keras.layers.Input(shape=(10, 469, 1)))
    
    hp_filters0 = hp.Int('filters0', min_value=4, max_value=16, step=4)
    
    hp_kernal0 = hp.Int('kernal0', min_value=2, max_value=4, step=1)
    
    hp_pool0 = hp.Int("pool0", min_value=2, max_value=4, step=1)
    
    hp_drop = hp.Float("drop", min_value=0, max_value=0.5, step=0.1)
    
    model.add(keras.layers.Conv2D(filters=hp_filters0, kernel_size=(hp_kernal0, 1), activation="relu"))
    
    model.add(keras.layers.MaxPool2D(pool_size=hp_pool0))
    
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dropout(hp_drop))

    model.add(keras.layers.Dense(7))

    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss=keras.losses.CategoricalHinge(),
                metrics=['accuracy'])

    return model

In [453]:
tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=50,
                     factor=3)
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
tuner.search(trainX, trainY, epochs=50, validation_split=0.2, callbacks=[stop_early])
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

Trial 90 Complete [00h 00m 15s]
val_accuracy: 0.7020958065986633

Best val_accuracy So Far: 0.7305389046669006
Total elapsed time: 00h 05m 16s
INFO:tensorflow:Oracle triggered exit


In [455]:
model = tuner.hypermodel.build(best_hps)
history = model.fit(trainX, trainY, epochs=100, validation_split=0.2)

val_acc = history.history['val_accuracy']
best_epoch = val_acc.index(max(val_acc))+1
print(best_epoch)

Epoch 1/100
84/84 [==============================] - 1s 6ms/step - loss: 20.8254 - accuracy: 0.2657 - val_loss: 8.5675 - val_accuracy: 0.4192
Epoch 2/100
84/84 [==============================] - 0s 4ms/step - loss: 15.3159 - accuracy: 0.3469 - val_loss: 5.7459 - val_accuracy: 0.5554
Epoch 3/100
84/84 [==============================] - 0s 4ms/step - loss: 12.7669 - accuracy: 0.4180 - val_loss: 4.5999 - val_accuracy: 0.6048
Epoch 4/100
84/84 [==============================] - 0s 4ms/step - loss: 11.0383 - accuracy: 0.4416 - val_loss: 4.9716 - val_accuracy: 0.5973
Epoch 5/100
84/84 [==============================] - 0s 4ms/step - loss: 9.6605 - accuracy: 0.4966 - val_loss: 4.1002 - val_accuracy: 0.6362
Epoch 6/100
84/84 [==============================] - 0s 4ms/step - loss: 9.3948 - accuracy: 0.5094 - val_loss: 4.6834 - val_accuracy: 0.6317
Epoch 7/100
84/84 [==============================] - 0s 4ms/step - loss: 8.4370 - accuracy: 0.5371 - val_loss: 3.8194 - val_accuracy: 0.6662
Epoch 8/1

Epoch 59/100
84/84 [==============================] - 0s 4ms/step - loss: 0.8341 - accuracy: 0.7803 - val_loss: 2.5301 - val_accuracy: 0.6931
Epoch 60/100
84/84 [==============================] - 0s 4ms/step - loss: 0.8358 - accuracy: 0.7803 - val_loss: 2.7643 - val_accuracy: 0.6946
Epoch 61/100
84/84 [==============================] - 0s 4ms/step - loss: 0.8801 - accuracy: 0.7721 - val_loss: 2.5973 - val_accuracy: 0.6916
Epoch 62/100
84/84 [==============================] - 0s 4ms/step - loss: 0.8789 - accuracy: 0.7653 - val_loss: 2.7301 - val_accuracy: 0.6946
Epoch 63/100
84/84 [==============================] - 0s 4ms/step - loss: 0.8080 - accuracy: 0.7751 - val_loss: 2.5423 - val_accuracy: 0.6961
Epoch 64/100
84/84 [==============================] - 0s 4ms/step - loss: 0.7479 - accuracy: 0.7822 - val_loss: 2.3985 - val_accuracy: 0.7036
Epoch 65/100
84/84 [==============================] - 0s 4ms/step - loss: 0.7874 - accuracy: 0.7698 - val_loss: 2.3809 - val_accuracy: 0.7036
Epoch 

In [456]:
hypermodel = tuner.hypermodel.build(best_hps)

hypermodel.fit(trainX, trainY, epochs=best_epoch, validation_split=0.2)

Epoch 1/34
84/84 [==============================] - 1s 5ms/step - loss: 22.3485 - accuracy: 0.2526 - val_loss: 6.4684 - val_accuracy: 0.4371
Epoch 2/34
84/84 [==============================] - 0s 4ms/step - loss: 15.2386 - accuracy: 0.3428 - val_loss: 4.9371 - val_accuracy: 0.5449
Epoch 3/34
84/84 [==============================] - 0s 4ms/step - loss: 12.9502 - accuracy: 0.4004 - val_loss: 4.8679 - val_accuracy: 0.5958
Epoch 4/34
84/84 [==============================] - 0s 4ms/step - loss: 11.2782 - accuracy: 0.4484 - val_loss: 4.3266 - val_accuracy: 0.6243
Epoch 5/34
84/84 [==============================] - 0s 4ms/step - loss: 9.8335 - accuracy: 0.4772 - val_loss: 4.4730 - val_accuracy: 0.6198
Epoch 6/34
84/84 [==============================] - 0s 4ms/step - loss: 9.7659 - accuracy: 0.5075 - val_loss: 3.9507 - val_accuracy: 0.6272
Epoch 7/34
84/84 [==============================] - 0s 4ms/step - loss: 8.3680 - accuracy: 0.5258 - val_loss: 3.8785 - val_accuracy: 0.6572
Epoch 8/34
84/84

In [457]:
eval_result = hypermodel.evaluate(testX, testY)
print("[test loss, test accuracy]:", eval_result)

27/27 [==============================] - 0s 2ms/step - loss: 3.0016 - accuracy: 0.6695
[test loss, test accuracy]: [3.001553535461426, 0.6694610714912415]


### Trying some more models

In [458]:
best_hps.get('filters0')

12

##### This was maxed, lets try increasing the max

In [459]:
best_hps.get('kernal0')

4

In [460]:
best_hps.get("pool0")

2

In [461]:
best_hps.get("drop")

0.4

In [508]:
best_hps.get("learning_rate")

0.0001

In [470]:
trainX.shape[1]

10

In [485]:
steps = trainX.shape[1]
features = trainX.shape[2]

In [487]:
features

469

In [509]:
def model_builder(hp):
    model = keras.Sequential()
    model.add(tf.keras.layers.Input(shape=(10, 469, 1)))
    
    hp_kernal0 = hp.Int('kernal0', min_value=4, max_value=8, step=1)
    
    #hp_lstm = hp.Int("lstm", min_value=64, max_value=256, step=64)
    #hp_drop0 = hp.Float("drop0", min_value=0, max_value=0.5, step=0.1)
    
    model.add(keras.layers.Conv2D(filters=12, kernel_size=(5, 1), activation="relu"))
    
    model.add(keras.layers.MaxPool2D(pool_size=2))
    
    model.add(keras.layers.Flatten())
    #model.add(tf.keras.layers.Reshape(target_shape=(18, 468)))
    #model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(hp_lstm, dropout=hp_drop0)))
    model.add(keras.layers.Dropout(0.4))

    model.add(keras.layers.Dense(7))

    model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001),
                loss=keras.losses.CategoricalHinge(),
                metrics=['accuracy'])

    return model

In [510]:
tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=50,
                     factor=3)
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
tuner.search(trainX, trainY, epochs=50, validation_split=0.2, callbacks=[stop_early])
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

Trial 84 Complete [00h 00m 18s]
val_accuracy: 0.7290419340133667

Best val_accuracy So Far: 0.735029935836792
Total elapsed time: 00h 05m 09s
INFO:tensorflow:Oracle triggered exit


In [513]:
best_hps.get("kernal0")

5

In [523]:
def model_builder(hp):
    model = keras.Sequential()
    model.add(tf.keras.layers.Input(shape=(10, 469, 1)))

    model.add(keras.layers.Conv2D(filters=12, kernel_size=(5, 1), activation="relu"))
    
    model.add(keras.layers.MaxPool2D(pool_size=2))
    
    model.add(keras.layers.Flatten())

    hp_units = hp.Int('units', min_value=32, max_value=128, step=32)
    hp_units1 = hp.Int('units1', min_value=64, max_value=256, step=64)
    hp_units2 = hp.Int('units2', min_value=64, max_value=256, step=64)
    hp_units3 = hp.Int('units3', min_value=64, max_value=256, step=64)
    hp_units4 = hp.Int('units4', min_value=64, max_value=256, step=64)
    
    model.add(keras.layers.Dense(units=hp_units, activation='relu'))
    model.add(keras.layers.Dense(units=hp_units1, activation='relu'))
    model.add(keras.layers.Dense(units=hp_units2, activation='relu'))
    model.add(keras.layers.Dense(units=hp_units3, activation='relu'))
    model.add(keras.layers.Dense(units=hp_units4, activation='relu'))
    
    model.add(keras.layers.Dropout(0.4))

    model.add(keras.layers.Dense(7))

    model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001),
                loss=keras.losses.CategoricalHinge(),
                metrics=['accuracy'])

    return model

In [524]:
tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=50,
                     factor=3)
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
tuner.search(trainX, trainY, epochs=50, validation_split=0.2, callbacks=[stop_early])
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

Trial 90 Complete [00h 00m 10s]
val_accuracy: 0.4940119683742523

Best val_accuracy So Far: 0.7215569019317627
Total elapsed time: 00h 06m 53s
INFO:tensorflow:Oracle triggered exit


In [525]:
model = tuner.hypermodel.build(best_hps)
history = model.fit(trainX, trainY, epochs=100, validation_split=0.2)

val_acc = history.history['val_accuracy']
best_epoch = val_acc.index(max(val_acc))+1
print(best_epoch)

Epoch 1/100
84/84 [==============================] - 1s 7ms/step - loss: 2.1123 - accuracy: 0.2631 - val_loss: 0.9532 - val_accuracy: 0.4611
Epoch 2/100
84/84 [==============================] - 0s 5ms/step - loss: 1.0690 - accuracy: 0.4207 - val_loss: 0.7598 - val_accuracy: 0.5479
Epoch 3/100
84/84 [==============================] - 0s 5ms/step - loss: 0.8449 - accuracy: 0.5161 - val_loss: 0.7091 - val_accuracy: 0.5464
Epoch 4/100
84/84 [==============================] - 0s 5ms/step - loss: 0.7077 - accuracy: 0.5865 - val_loss: 0.5461 - val_accuracy: 0.6452
Epoch 5/100
84/84 [==============================] - 0s 5ms/step - loss: 0.5740 - accuracy: 0.6396 - val_loss: 0.5363 - val_accuracy: 0.6617
Epoch 6/100
84/84 [==============================] - 0s 5ms/step - loss: 0.5234 - accuracy: 0.6714 - val_loss: 0.4739 - val_accuracy: 0.6707
Epoch 7/100
84/84 [==============================] - 0s 6ms/step - loss: 0.5164 - accuracy: 0.6699 - val_loss: 0.4393 - val_accuracy: 0.6737
Epoch 8/100
8

Epoch 59/100
84/84 [==============================] - 0s 5ms/step - loss: 0.0873 - accuracy: 0.8436 - val_loss: 0.5645 - val_accuracy: 0.6976
Epoch 60/100
84/84 [==============================] - 0s 5ms/step - loss: 0.0492 - accuracy: 0.8533 - val_loss: 0.5572 - val_accuracy: 0.7156
Epoch 61/100
84/84 [==============================] - 0s 5ms/step - loss: 0.0405 - accuracy: 0.8608 - val_loss: 0.5543 - val_accuracy: 0.7156
Epoch 62/100
84/84 [==============================] - 0s 5ms/step - loss: 0.0326 - accuracy: 0.8619 - val_loss: 0.5895 - val_accuracy: 0.6961
Epoch 63/100
84/84 [==============================] - 0s 6ms/step - loss: 0.0288 - accuracy: 0.8656 - val_loss: 0.5333 - val_accuracy: 0.7081
Epoch 64/100
84/84 [==============================] - 0s 6ms/step - loss: 0.0418 - accuracy: 0.8589 - val_loss: 0.5653 - val_accuracy: 0.7081
Epoch 65/100
84/84 [==============================] - 0s 5ms/step - loss: 0.0243 - accuracy: 0.8653 - val_loss: 0.5857 - val_accuracy: 0.6991
Epoch 

In [526]:
hypermodel = tuner.hypermodel.build(best_hps)

hypermodel.fit(trainX, trainY, epochs=best_epoch, validation_split=0.2)

Epoch 1/40
84/84 [==============================] - 1s 7ms/step - loss: 1.6358 - accuracy: 0.2814 - val_loss: 0.9049 - val_accuracy: 0.4237
Epoch 2/40
84/84 [==============================] - 0s 6ms/step - loss: 0.9496 - accuracy: 0.3885 - val_loss: 0.8358 - val_accuracy: 0.4371
Epoch 3/40
84/84 [==============================] - 0s 5ms/step - loss: 0.7926 - accuracy: 0.4865 - val_loss: 0.6438 - val_accuracy: 0.5689
Epoch 4/40
84/84 [==============================] - 0s 5ms/step - loss: 0.6237 - accuracy: 0.5767 - val_loss: 0.5448 - val_accuracy: 0.5868
Epoch 5/40
84/84 [==============================] - 0s 6ms/step - loss: 0.5369 - accuracy: 0.6063 - val_loss: 0.5351 - val_accuracy: 0.5958
Epoch 6/40
84/84 [==============================] - 0s 5ms/step - loss: 0.4904 - accuracy: 0.6284 - val_loss: 0.4886 - val_accuracy: 0.6063
Epoch 7/40
84/84 [==============================] - 0s 5ms/step - loss: 0.4383 - accuracy: 0.6415 - val_loss: 0.4592 - val_accuracy: 0.6392
Epoch 8/40
84/84 [==

In [527]:
eval_result = hypermodel.evaluate(testX, testY)
print("[test loss, test accuracy]:", eval_result)

27/27 [==============================] - 0s 2ms/step - loss: 0.5507 - accuracy: 0.6623
[test loss, test accuracy]: [0.5506706237792969, 0.6622754335403442]


### Best performing model

In [936]:
def model_builder(hp):
    model = keras.Sequential()
    model.add(tf.keras.layers.Flatten(input_shape=(1, 1, 469, 1)))
    
    hp_units = hp.Int('units', min_value=32, max_value=128, step=32)
    hp_units1 = hp.Int('units1', min_value=64, max_value=256, step=64)
    hp_units2 = hp.Int('units2', min_value=64, max_value=256, step=64)
    hp_units3 = hp.Int('units3', min_value=64, max_value=256, step=64)
    hp_units4 = hp.Int('units4', min_value=64, max_value=256, step=64)
    
    model.add(keras.layers.Dense(units=hp_units, activation='relu'))
    model.add(keras.layers.Dense(units=hp_units1, activation='relu'))
    model.add(keras.layers.Dense(units=hp_units2, activation='relu'))
    model.add(keras.layers.Dense(units=hp_units3, activation='relu'))
    model.add(keras.layers.Dense(units=hp_units4, activation='relu'))
    
    hp_drop = hp.Float('drop', min_value=0, max_value=0.5, step=0.1)
    
    model.add(tf.keras.layers.Dropout(rate=hp_drop),)
    
    model.add(keras.layers.Dense(7))

    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss=keras.losses.CategoricalHinge(),
                metrics=['accuracy'])

    return model

In [1004]:
model = keras.Sequential()
model.add(tf.keras.layers.Flatten(input_shape=(1, 10, 469, 1)))

model.add(keras.layers.Dense(units=fcbest_hps.get("units"), activation='relu'))
model.add(keras.layers.Dense(units=fcbest_hps.get("units1"), activation='relu'))
model.add(keras.layers.Dense(units=fcbest_hps.get("units2"), activation='relu'))
model.add(keras.layers.Dense(units=fcbest_hps.get("units3"), activation='relu'))
model.add(keras.layers.Dense(units=fcbest_hps.get("units4"), activation='relu'))

model.add(tf.keras.layers.Dropout(rate=fcbest_hps.get("drop")),)
    
model.add(keras.layers.Dense(7))

model.compile(optimizer=keras.optimizers.Adam(learning_rate=fcbest_hps.get("learning_rate")),
                loss=keras.losses.CategoricalHinge(),
                metrics=['accuracy'])
print(model.summary())

Model: "sequential_34"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_33 (Flatten)        (None, 4690)              0         
                                                                 
 dense_163 (Dense)           (None, 96)                450336    
                                                                 
 dense_164 (Dense)           (None, 64)                6208      
                                                                 
 dense_165 (Dense)           (None, 192)               12480     
                                                                 
 dense_166 (Dense)           (None, 192)               37056     
                                                                 
 dense_167 (Dense)           (None, 64)                12352     
                                                                 
 dropout_30 (Dropout)        (None, 64)              

In [1005]:
smodel = keras.Sequential()
smodel.add(tf.keras.layers.Flatten(input_shape=(1, 469, 1)))

smodel.add(keras.layers.Dense(units=fcbest_hps.get("units"), activation='relu'))
smodel.add(keras.layers.Dense(units=fcbest_hps.get("units1"), activation='relu'))
smodel.add(keras.layers.Dense(units=fcbest_hps.get("units2"), activation='relu'))
smodel.add(keras.layers.Dense(units=fcbest_hps.get("units3"), activation='relu'))
smodel.add(keras.layers.Dense(units=fcbest_hps.get("units4"), activation='relu'))

smodel.add(tf.keras.layers.Dropout(rate=fcbest_hps.get("drop")),)
    
smodel.add(keras.layers.Dense(7))

smodel.compile(optimizer=keras.optimizers.Adam(learning_rate=fcbest_hps.get("learning_rate")),
                loss=keras.losses.CategoricalHinge(),
                metrics=['accuracy'])
print(smodel.summary())

Model: "sequential_35"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_34 (Flatten)        (None, 469)               0         
                                                                 
 dense_169 (Dense)           (None, 96)                45120     
                                                                 
 dense_170 (Dense)           (None, 64)                6208      
                                                                 
 dense_171 (Dense)           (None, 192)               12480     
                                                                 
 dense_172 (Dense)           (None, 192)               37056     
                                                                 
 dense_173 (Dense)           (None, 64)                12352     
                                                                 
 dropout_31 (Dropout)        (None, 64)              

In [1006]:
Y = big_data[["Electronic", "Experimental", "Folk", "Hip-Hop", "International", "Punk", "Rock"]]
X = big_data[["zcross", "mfcc", "chroma", "spectral"]]
X.loc[:,'chroma'] = X['chroma'].apply(lambda x: x.reshape(-1,10,469,1))
chroma = np.array(X['chroma'].tolist())

X.loc[:,'mfcc'] = X['mfcc'].apply(lambda x: x.reshape(-1,10,469,1))
mfcc= np.array(X['mfcc'].tolist())

X.loc[:,'spectral'] = X['spectral'].apply(lambda x: x.reshape(-1,469,1))
spectral= np.array(X['spectral'].tolist())

X.loc[:,'zcross'] = X['zcross'].apply(lambda x: x.reshape(-1,469,1))
zcross= np.array(X['zcross'].tolist())

In [1015]:
trainX, testX, trainY, testY = train_test_split(chroma, Y, stratify=Y, test_size=0.2)
model.fit(trainX, trainY, validation_data=(testX, testY), epochs=60, batch_size=64)

Epoch 1/60
53/53 [==============================] - 1s 9ms/step - loss: 0.0181 - accuracy: 0.8677 - val_loss: 0.0276 - val_accuracy: 0.8647
Epoch 2/60
53/53 [==============================] - 0s 6ms/step - loss: 0.0291 - accuracy: 0.8632 - val_loss: 0.0132 - val_accuracy: 0.8695
Epoch 3/60
53/53 [==============================] - 0s 6ms/step - loss: 0.0187 - accuracy: 0.8671 - val_loss: 0.0215 - val_accuracy: 0.8647
Epoch 4/60
53/53 [==============================] - 0s 6ms/step - loss: 0.0479 - accuracy: 0.8560 - val_loss: 0.0314 - val_accuracy: 0.8623
Epoch 5/60
53/53 [==============================] - 0s 6ms/step - loss: 0.0111 - accuracy: 0.8713 - val_loss: 0.0106 - val_accuracy: 0.8683
Epoch 6/60
53/53 [==============================] - 0s 6ms/step - loss: 0.0038 - accuracy: 0.8722 - val_loss: 0.0257 - val_accuracy: 0.8623
Epoch 7/60
53/53 [==============================] - 0s 6ms/step - loss: 0.0049 - accuracy: 0.8719 - val_loss: 0.0144 - val_accuracy: 0.8671
Epoch 8/60
53/53 [==

Epoch 59/60
53/53 [==============================] - 0s 4ms/step - loss: 0.0197 - accuracy: 0.8671 - val_loss: 0.0324 - val_accuracy: 0.8611
Epoch 60/60
53/53 [==============================] - 0s 4ms/step - loss: 0.0205 - accuracy: 0.8659 - val_loss: 0.0332 - val_accuracy: 0.8611


### Lets try the different features

## Chroma

In [1016]:
eval_result = model.evaluate(testX, testY)
print("[test loss, test accuracy]:", eval_result)

27/27 [==============================] - 0s 2ms/step - loss: 0.0332 - accuracy: 0.8611
[test loss, test accuracy]: [0.033224258571863174, 0.8610778450965881]


In [1017]:
trainX, testX, trainY, testY = train_test_split(mfcc, Y, stratify=Y, test_size=0.2)
model.fit(trainX, trainY, validation_data=(testX, testY), epochs=60, batch_size=64)

Epoch 1/60
53/53 [==============================] - 1s 10ms/step - loss: 44.4344 - accuracy: 0.3731 - val_loss: 2.1137 - val_accuracy: 0.5389
Epoch 2/60
53/53 [==============================] - 0s 6ms/step - loss: 1.2300 - accuracy: 0.4138 - val_loss: 0.9581 - val_accuracy: 0.4731
Epoch 3/60
53/53 [==============================] - 0s 6ms/step - loss: 0.8646 - accuracy: 0.4216 - val_loss: 0.8393 - val_accuracy: 0.4575
Epoch 4/60
53/53 [==============================] - 0s 6ms/step - loss: 0.7878 - accuracy: 0.4210 - val_loss: 0.8358 - val_accuracy: 0.4311
Epoch 5/60
53/53 [==============================] - 0s 6ms/step - loss: 0.7619 - accuracy: 0.4156 - val_loss: 0.7808 - val_accuracy: 0.4802
Epoch 6/60
53/53 [==============================] - 0s 5ms/step - loss: 0.7276 - accuracy: 0.4251 - val_loss: 0.7818 - val_accuracy: 0.4587
Epoch 7/60
53/53 [==============================] - 0s 4ms/step - loss: 0.7064 - accuracy: 0.4246 - val_loss: 0.7716 - val_accuracy: 0.3928
Epoch 8/60
53/53 [

53/53 [==============================] - 0s 5ms/step - loss: 0.4760 - accuracy: 0.5491 - val_loss: 0.7672 - val_accuracy: 0.5030
Epoch 60/60
53/53 [==============================] - 0s 5ms/step - loss: 0.4676 - accuracy: 0.5698 - val_loss: 0.8099 - val_accuracy: 0.4778


## MFCC

In [1018]:
eval_result = model.evaluate(testX, testY)
print("[test loss, test accuracy]:", eval_result)

27/27 [==============================] - 0s 2ms/step - loss: 0.8098 - accuracy: 0.4778
[test loss, test accuracy]: [0.8098351955413818, 0.4778442978858948]


In [1019]:
model = keras.Sequential()
model.add(tf.keras.layers.Flatten(input_shape=(1, 469, 1)))

model.add(keras.layers.Dense(units=best_hps.get("units"), activation='relu'))
model.add(keras.layers.Dense(units=best_hps.get("units1"), activation='relu'))
model.add(keras.layers.Dense(units=best_hps.get("units2"), activation='relu'))
model.add(keras.layers.Dense(units=best_hps.get("units3"), activation='relu'))
model.add(keras.layers.Dense(units=best_hps.get("units4"), activation='relu'))

model.add(tf.keras.layers.Dropout(rate=best_hps.get("drop")),)
    
model.add(keras.layers.Dense(7))

model.compile(optimizer=keras.optimizers.Adam(learning_rate=best_hps.get("learning_rate")),
                loss=keras.losses.CategoricalHinge(),
                metrics=['accuracy'])
print(model.summary())
trainX, testX, trainY, testY = train_test_split(spectral, Y, stratify=Y, test_size=0.2)
model.fit(trainX, trainY, validation_data=(testX, testY), epochs=60, batch_size=64)

Model: "sequential_36"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_35 (Flatten)        (None, 469)               0         
                                                                 
 dense_175 (Dense)           (None, 96)                45120     
                                                                 
 dense_176 (Dense)           (None, 64)                6208      
                                                                 
 dense_177 (Dense)           (None, 192)               12480     
                                                                 
 dense_178 (Dense)           (None, 192)               37056     
                                                                 
 dense_179 (Dense)           (None, 64)                12352     
                                                                 
 dropout_32 (Dropout)        (None, 64)              

53/53 [==============================] - 0s 5ms/step - loss: 0.9951 - accuracy: 0.2072 - val_loss: 1.4350 - val_accuracy: 0.2551
Epoch 49/60
53/53 [==============================] - 0s 5ms/step - loss: 0.9791 - accuracy: 0.2132 - val_loss: 1.4408 - val_accuracy: 0.1257
Epoch 50/60
53/53 [==============================] - 0s 5ms/step - loss: 0.9870 - accuracy: 0.2033 - val_loss: 1.4359 - val_accuracy: 0.1257
Epoch 51/60
53/53 [==============================] - 0s 5ms/step - loss: 0.9814 - accuracy: 0.1976 - val_loss: 1.4369 - val_accuracy: 0.2551
Epoch 52/60
53/53 [==============================] - 0s 5ms/step - loss: 0.9803 - accuracy: 0.1964 - val_loss: 1.4483 - val_accuracy: 0.2563
Epoch 53/60
53/53 [==============================] - 0s 5ms/step - loss: 0.9831 - accuracy: 0.2024 - val_loss: 1.4351 - val_accuracy: 0.2551
Epoch 54/60
53/53 [==============================] - 0s 5ms/step - loss: 0.9850 - accuracy: 0.2180 - val_loss: 1.4192 - val_accuracy: 0.2563
Epoch 55/60
53/53 [======

## Spectral

In [1020]:
eval_result = model.evaluate(testX, testY)
print("[test loss, test accuracy]:", eval_result)

27/27 [==============================] - 0s 4ms/step - loss: 1.4342 - accuracy: 0.2539
[test loss, test accuracy]: [1.4342151880264282, 0.2538922131061554]


In [1021]:
model = keras.Sequential()
model.add(tf.keras.layers.Flatten(input_shape=(2, 10, 469, 1)))

model.add(keras.layers.Dense(units=fcbest_hps.get("units"), activation='relu'))
model.add(keras.layers.Dense(units=fcbest_hps.get("units1"), activation='relu'))
model.add(keras.layers.Dense(units=fcbest_hps.get("units2"), activation='relu'))
model.add(keras.layers.Dense(units=fcbest_hps.get("units3"), activation='relu'))
model.add(keras.layers.Dense(units=fcbest_hps.get("units4"), activation='relu'))

model.add(tf.keras.layers.Dropout(rate=fcbest_hps.get("drop")),)
    
model.add(keras.layers.Dense(7))

model.compile(optimizer=keras.optimizers.Adam(learning_rate=fcbest_hps.get("learning_rate")),
                loss=keras.losses.CategoricalHinge(),
                metrics=['accuracy'])
chromamfcc = np.concatenate((chroma,mfcc),axis=1)

trainX, testX, trainY, testY = train_test_split(chromamfcc, Y, stratify=Y, test_size=0.2)
model.fit(trainX, trainY, validation_data=(testX, testY), epochs=60, batch_size=64)

Epoch 1/60
53/53 [==============================] - 1s 11ms/step - loss: 3.6185 - accuracy: 0.3431 - val_loss: 1.0803 - val_accuracy: 0.5281
Epoch 2/60
53/53 [==============================] - 0s 4ms/step - loss: 1.2643 - accuracy: 0.5090 - val_loss: 0.7372 - val_accuracy: 0.6431
Epoch 3/60
53/53 [==============================] - 0s 4ms/step - loss: 0.8781 - accuracy: 0.5958 - val_loss: 0.6550 - val_accuracy: 0.6683
Epoch 4/60
53/53 [==============================] - 0s 5ms/step - loss: 0.6951 - accuracy: 0.6362 - val_loss: 0.5898 - val_accuracy: 0.6754
Epoch 5/60
53/53 [==============================] - 0s 5ms/step - loss: 0.6048 - accuracy: 0.6620 - val_loss: 0.5807 - val_accuracy: 0.6802
Epoch 6/60
53/53 [==============================] - 0s 4ms/step - loss: 0.5177 - accuracy: 0.6895 - val_loss: 0.5525 - val_accuracy: 0.6946
Epoch 7/60
53/53 [==============================] - 0s 4ms/step - loss: 0.4707 - accuracy: 0.6994 - val_loss: 0.5077 - val_accuracy: 0.6958
Epoch 8/60
53/53 [=

53/53 [==============================] - 0s 5ms/step - loss: 0.0644 - accuracy: 0.8515 - val_loss: 0.6532 - val_accuracy: 0.7054
Epoch 60/60
53/53 [==============================] - 0s 5ms/step - loss: 0.0386 - accuracy: 0.8626 - val_loss: 0.6644 - val_accuracy: 0.7150


## Chroma + MFCC

In [1022]:
eval_result = model.evaluate(testX, testY)
print("[test loss, test accuracy]:", eval_result)

27/27 [==============================] - 0s 2ms/step - loss: 0.6644 - accuracy: 0.7150
[test loss, test accuracy]: [0.6643737554550171, 0.714970052242279]


In [1023]:
trainX, testX, trainY, testY = train_test_split(zcross, Y, stratify=Y, test_size=0.2)
smodel.fit(trainX, trainY, validation_data=(testX, testY), epochs=60, batch_size=64)

Epoch 1/60
53/53 [==============================] - 1s 9ms/step - loss: 1.0137 - accuracy: 0.2320 - val_loss: 0.9991 - val_accuracy: 0.3257
Epoch 2/60
53/53 [==============================] - 0s 7ms/step - loss: 0.9972 - accuracy: 0.3674 - val_loss: 0.9933 - val_accuracy: 0.4251
Epoch 3/60
53/53 [==============================] - 0s 5ms/step - loss: 0.9909 - accuracy: 0.4344 - val_loss: 0.9861 - val_accuracy: 0.4527
Epoch 4/60
53/53 [==============================] - 0s 5ms/step - loss: 0.9808 - accuracy: 0.4467 - val_loss: 0.9745 - val_accuracy: 0.4611
Epoch 5/60
53/53 [==============================] - 0s 5ms/step - loss: 0.9646 - accuracy: 0.4665 - val_loss: 0.9598 - val_accuracy: 0.4299
Epoch 6/60
53/53 [==============================] - 0s 5ms/step - loss: 0.9385 - accuracy: 0.4614 - val_loss: 0.9232 - val_accuracy: 0.4527
Epoch 7/60
53/53 [==============================] - 0s 5ms/step - loss: 0.8817 - accuracy: 0.4641 - val_loss: 0.8473 - val_accuracy: 0.4467
Epoch 8/60
53/53 [==

53/53 [==============================] - 0s 5ms/step - loss: 0.2706 - accuracy: 0.7530 - val_loss: 0.5020 - val_accuracy: 0.6467
Epoch 60/60
53/53 [==============================] - 0s 5ms/step - loss: 0.2633 - accuracy: 0.7500 - val_loss: 0.4983 - val_accuracy: 0.6515


## Zcross

In [1024]:
eval_result = smodel.evaluate(testX, testY)
print("[test loss, test accuracy]:", eval_result)

27/27 [==============================] - 0s 2ms/step - loss: 0.4983 - accuracy: 0.6515
[test loss, test accuracy]: [0.49834349751472473, 0.6514970064163208]


In [1025]:
model = keras.Sequential()
model.add(tf.keras.layers.Flatten(input_shape=(2, 469, 1)))

model.add(keras.layers.Dense(units=fcbest_hps.get("units"), activation='relu'))
model.add(keras.layers.Dense(units=fcbest_hps.get("units1"), activation='relu'))
model.add(keras.layers.Dense(units=fcbest_hps.get("units2"), activation='relu'))
model.add(keras.layers.Dense(units=fcbest_hps.get("units3"), activation='relu'))
model.add(keras.layers.Dense(units=fcbest_hps.get("units4"), activation='relu'))

model.add(tf.keras.layers.Dropout(rate=fcbest_hps.get("drop")),)
    
model.add(keras.layers.Dense(7))

model.compile(optimizer=keras.optimizers.Adam(learning_rate=fcbest_hps.get("learning_rate")),
                loss=keras.losses.CategoricalHinge(),
                metrics=['accuracy'])
spectralcross = np.concatenate((zcross,spectral),axis=1)

trainX, testX, trainY, testY = train_test_split(spectralcross, Y, stratify=Y, test_size=0.2)
model.fit(trainX, trainY, validation_data=(testX, testY), epochs=60, batch_size=64)

Epoch 1/60
53/53 [==============================] - 1s 9ms/step - loss: 168.7832 - accuracy: 0.1877 - val_loss: 32.2623 - val_accuracy: 0.2515
Epoch 2/60
53/53 [==============================] - 0s 7ms/step - loss: 37.2635 - accuracy: 0.2488 - val_loss: 19.1136 - val_accuracy: 0.2814
Epoch 3/60
53/53 [==============================] - 0s 6ms/step - loss: 19.3978 - accuracy: 0.2907 - val_loss: 11.1170 - val_accuracy: 0.3353
Epoch 4/60
53/53 [==============================] - 0s 5ms/step - loss: 10.6378 - accuracy: 0.2925 - val_loss: 6.9922 - val_accuracy: 0.3257
Epoch 5/60
53/53 [==============================] - 0s 5ms/step - loss: 4.7520 - accuracy: 0.3284 - val_loss: 3.5512 - val_accuracy: 0.2778
Epoch 6/60
53/53 [==============================] - 0s 5ms/step - loss: 2.2093 - accuracy: 0.2889 - val_loss: 2.2478 - val_accuracy: 0.2838
Epoch 7/60
53/53 [==============================] - 0s 5ms/step - loss: 1.5878 - accuracy: 0.2467 - val_loss: 1.8547 - val_accuracy: 0.2778
Epoch 8/60
5

53/53 [==============================] - 0s 4ms/step - loss: 0.9788 - accuracy: 0.2066 - val_loss: 1.1744 - val_accuracy: 0.2587
Epoch 60/60
53/53 [==============================] - 0s 4ms/step - loss: 0.9790 - accuracy: 0.2066 - val_loss: 1.1667 - val_accuracy: 0.2587


## ZCross+Spectral

In [1026]:
eval_result = model.evaluate(testX, testY)
print("[test loss, test accuracy]:", eval_result)

27/27 [==============================] - 0s 2ms/step - loss: 1.1666 - accuracy: 0.2587
[test loss, test accuracy]: [1.1665539741516113, 0.25868263840675354]


### Chroma alone performs the best on a fully connect neural net.